In [11]:
import os
import cv2
import time
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from scipy import misc
from Detection import Detection
from Recognition import Recognition
import preprocess
import imutils
import math

In [12]:
#Path for reading the video
Input_video_path = "../../EximiusAI_Dataset/Original/Videos/20190307_171714.mp4"
Output_video_path = "../Outputs/Detection_Recognition_outputs/"
Output_video_name = "EximiusAI_SGD_align.avi"
classifier_pickle_file = "../util/EximiusAI_J.pkl"
O_path = Output_video_path + Output_video_name

In [13]:
#Path for the detection and recognition pb files
Detection_model_path = "../Models/Detection_mtcnn.pb"
Recognition_model_path = "../Models/Recognition_facenet.pb"

In [14]:
#Instances of detection and recognition are being created.
#Instances are created to avoid loading the graphs and sessions again and again for every frame.
detection = Detection(Detection_model_path)
recognition = Recognition(Recognition_model_path)

Detection Model Graph Initialized
Recognition Model Graph Initialized


In [15]:
with open(classifier_pickle_file, 'rb') as infile:
    model,class_names = pickle.load(infile)

In [17]:
#Initializing video capture from the Input_video_path.
cap = cv2.VideoCapture(Input_video_path)
#Variable to count frames.
frame_count = 0
#starting the processing time to calculate fps.
start = time.time()
#Ensuring the input_video_path opens without errors.
if (cap.isOpened()== False):
    print("Error opening video stream or file")

#getting the frame_width , frame_height from the given input video.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_area = frame_width * frame_height

#creating a video file to write the output frames at output_video_path(O_path).

out = cv2.VideoWriter(O_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30 , (frame_width,frame_height))

#Reading each and every frame in a while loop and processing/inferencing them through two models.
grab = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    frame_start_time = time.time()
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_count = frame_count+1
    if ret != True or frame_count>10000:
        break
    if ret == True:
        
        #Detection Starts :
        
        #Inferencing the Detection model
        bbox, scores, landmarks = detection.detect(frame)
        for box, pts in zip(bbox, landmarks):
            box = box.astype('int32')
            box_w = box[3] - box[1]
            box_h = box[2] - box[0]
            box_a = box_w*box_h
            percent = box_a*100/frame_area
            pts = pts.astype('int32')
            x0 = pts[5]
            y0 = pts[0]
            x1 = pts[6]
            y1 = pts[1]
            m = (y1-y0)/(x1-x0)
            deg = math.degrees(math.atan(m))
            frame1 = imutils.rotate(frame, (deg))
            
            # CROPPING THE FACES OUT OF THE IMAGE AND APPENDING THEM TO THE LIST
            print('[INFO] percentage of bounding box in total image : {:.2f}'.format(percent))
            face = np.copy(frame1[box[0]:box[2] , box[1]:box[3]])
            if percent >1.0 and face.shape[0] != 0 and face.shape[1]!= 0 and face.shape[2] !=0:
                if grab == 0:
                    img = face
                grab = grab+1
#                 plt.imshow(face)
#                 plt.show()
                face = preprocess.prewhiten(face)
#                 print(face.shape)
                embedding = recognition.recognize(face = face)
                predictions = model.predict_proba(embedding)
                prediction_id = model.predict(embedding)
                print("PREDICTIONS :",predictions)
                best_class_indices = np.argmax(predictions, axis=1)
                print("BEST CLASS INDICES", best_class_indices)
                best_class_probabilities = predictions[np.arange(len(best_class_indices)), best_class_indices]
                print(best_class_probabilities)
                tup = (0,255,0)
                conf = ""
                if best_class_probabilities > 0.8:
                    conf = "high"
                    tup = (0,255,0)
                if best_class_probabilities > 0.5 and best_class_probabilities < 0.8:
                    conf = "low"
                    tup = (0,255,255)
                if best_class_probabilities < 0.5:
                    prediction_id = ["Unknown"]
                    tup = (0,0,255)
                    conf = "-"
                print(str(prediction_id[0]))
#                 if str(prediction_id[0]) == "Santosh_Vasa":
#                     cv2.imwrite("/home/santoshv/SAVE/"+str(frame_count)+".jpg",frame)
#                 print("BEST CLASS PROBABILITIES", best_class_probabilities)
#                 for i in range(len(best_class_indices)):
#                     print('%4d  %s: %.3f' % (i, class_names[best_class_indices[i]], best_class_probabilities[i]))
                frame = cv2.rectangle(frame, (box[1], box[0]), (box[3], box[2]), tup, 3)
                pts = pts.astype('int32')
                
                class_prob = str(prediction_id[0])+" : {:.2f}".format(best_class_probabilities[0])
                cv2.putText(frame, class_prob, (box[1], box[0]),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, tup ,
                            thickness=2, lineType=2)
                cv2.putText(frame, conf, (box[3], box[0]),cv2.FONT_HERSHEY_SIMPLEX, 1, tup ,thickness=2, lineType=2)
                #Uncomment this to have landmarks
#                 for i in range(5):
#                     frame = cv2.circle(frame, (pts[i+5], pts[i]), 4, (0, 0, 255), 8)
                
        #frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        #Writing to the video output
        out.write(frame)
        
        frame_end_time = time.time()
        time_per_frame = frame_end_time - frame_start_time
        fps_frame = 1/time_per_frame
        print('[INFO] total boxes:', len(bbox))
        print('[INFO] Processing Frame:', frame_count)
        print('[INFO] Processing Speed:',fps_frame," FPS")
        print('[INFO] Time Per Frame:', time_per_frame)
        
end = time.time()
timet = end - start
fps = frame_count/timet
print("[INFO] NUMBER OF FRAMES:", frame_count)
print("[INFO] Detection took {:.5} seconds".format(end - start))
print("[INFO] Overall FPS: "+ str(fps))

# closing the writer and reader

cap.release()
out.release()

[INFO] total boxes: 0
[INFO] Processing Frame: 1
[INFO] Processing Speed: 20.468206794913087  FPS
[INFO] Time Per Frame: 0.048856258392333984
[INFO] total boxes: 0
[INFO] Processing Frame: 2
[INFO] Processing Speed: 21.07298642965881  FPS
[INFO] Time Per Frame: 0.047454118728637695
[INFO] total boxes: 0
[INFO] Processing Frame: 3
[INFO] Processing Speed: 19.486004450700822  FPS
[INFO] Time Per Frame: 0.05131888389587402
[INFO] total boxes: 0
[INFO] Processing Frame: 4
[INFO] Processing Speed: 19.66857523364705  FPS
[INFO] Time Per Frame: 0.0508425235748291
[INFO] total boxes: 0
[INFO] Processing Frame: 5
[INFO] Processing Speed: 20.786519972247003  FPS
[INFO] Time Per Frame: 0.04810810089111328
[INFO] total boxes: 0
[INFO] Processing Frame: 6
[INFO] Processing Speed: 20.73257704950446  FPS
[INFO] Time Per Frame: 0.0482332706451416
[INFO] total boxes: 0
[INFO] Processing Frame: 7
[INFO] Processing Speed: 19.877558564407817  FPS
[INFO] Time Per Frame: 0.0503079891204834
[INFO] total boxe

PREDICTIONS : [[4.49154717e-02 6.73897684e-03 6.48886462e-04 1.24507539e-02
  5.28386822e-03 9.60893324e-04 8.81254063e-01 1.96776837e-02
  5.98873150e-06 2.59894072e-04 2.78035197e-02]]
BEST CLASS INDICES [6]
[0.88125406]
Saikrishna_Dyavarasetti
[INFO] total boxes: 1
[INFO] Processing Frame: 34
[INFO] Processing Speed: 13.203960271363586  FPS
[INFO] Time Per Frame: 0.07573485374450684
[INFO] percentage of bounding box in total image : 2.82
PREDICTIONS : [[7.09426482e-02 2.81843658e-03 4.75242347e-04 1.99407545e-03
  2.59656780e-03 7.94272558e-04 8.54227547e-01 5.61802484e-02
  6.59669543e-06 1.72666894e-04 9.79169767e-03]]
BEST CLASS INDICES [6]
[0.85422755]
Saikrishna_Dyavarasetti
[INFO] total boxes: 1
[INFO] Processing Frame: 35
[INFO] Processing Speed: 13.33599567581317  FPS
[INFO] Time Per Frame: 0.07498502731323242
[INFO] percentage of bounding box in total image : 2.81
PREDICTIONS : [[8.87289967e-02 4.01289798e-03 6.87627849e-04 6.33303420e-03
  4.92883204e-03 2.00844005e-03 8.3

[INFO] total boxes: 1
[INFO] Processing Frame: 52
[INFO] Processing Speed: 12.581829962623214  FPS
[INFO] Time Per Frame: 0.07947969436645508
[INFO] percentage of bounding box in total image : 3.42
PREDICTIONS : [[2.52911207e-02 3.72703351e-02 5.90373511e-04 5.81028645e-03
  2.82576009e-03 1.67174042e-04 9.11874045e-01 2.88925207e-03
  1.75081214e-05 7.61721009e-05 1.31879727e-02]]
BEST CLASS INDICES [6]
[0.91187405]
Saikrishna_Dyavarasetti
[INFO] total boxes: 1
[INFO] Processing Frame: 53
[INFO] Processing Speed: 12.055195803693325  FPS
[INFO] Time Per Frame: 0.08295178413391113
[INFO] percentage of bounding box in total image : 3.39
PREDICTIONS : [[4.86033150e-02 1.40987918e-02 2.66801998e-04 6.54600185e-03
  9.22043197e-03 3.80036561e-04 8.87628037e-01 1.05635425e-02
  8.02247792e-06 2.58591006e-04 2.24264280e-02]]
BEST CLASS INDICES [6]
[0.88762804]
Saikrishna_Dyavarasetti
[INFO] total boxes: 1
[INFO] Processing Frame: 54
[INFO] Processing Speed: 12.539625213762093  FPS
[INFO] Time

[INFO] total boxes: 1
[INFO] Processing Frame: 72
[INFO] Processing Speed: 13.236754736988273  FPS
[INFO] Time Per Frame: 0.0755472183227539
[INFO] percentage of bounding box in total image : 3.94
PREDICTIONS : [[1.31571720e-01 2.38867455e-03 1.86865654e-04 1.92843977e-03
  3.93023801e-03 7.63629126e-04 8.14207776e-01 3.43899303e-02
  4.85376104e-06 3.50684839e-04 1.02771886e-02]]
BEST CLASS INDICES [6]
[0.81420778]
Saikrishna_Dyavarasetti
[INFO] total boxes: 1
[INFO] Processing Frame: 73
[INFO] Processing Speed: 12.004991699582117  FPS
[INFO] Time Per Frame: 0.0832986831665039
[INFO] percentage of bounding box in total image : 3.94
PREDICTIONS : [[1.18842400e-01 2.78336321e-03 1.31893072e-04 3.41935047e-04
  5.74643252e-03 4.58639832e-04 8.45218176e-01 7.80464954e-03
  1.80932111e-05 1.11058912e-03 1.75438287e-02]]
BEST CLASS INDICES [6]
[0.84521818]
Saikrishna_Dyavarasetti
[INFO] total boxes: 1
[INFO] Processing Frame: 74
[INFO] Processing Speed: 12.665911719497869  FPS
[INFO] Time P

[INFO] percentage of bounding box in total image : 5.36
PREDICTIONS : [[7.11896381e-02 1.22612616e-03 3.10153137e-04 1.15010354e-03
  6.74638998e-03 4.04242599e-03 8.80694129e-01 3.28205796e-02
  9.50444607e-05 2.94255002e-04 1.43115547e-03]]
BEST CLASS INDICES [6]
[0.88069413]
Saikrishna_Dyavarasetti
[INFO] percentage of bounding box in total image : 2.57
PREDICTIONS : [[8.67422761e-03 3.76779516e-02 2.39189386e-04 1.00894492e-01
  4.02198065e-01 5.78147414e-02 2.89074287e-01 4.13986579e-02
  2.86262567e-03 5.70880678e-02 2.07769402e-03]]
BEST CLASS INDICES [4]
[0.40219807]
Unknown
[INFO] total boxes: 2
[INFO] Processing Frame: 91
[INFO] Processing Speed: 9.30799599656468  FPS
[INFO] Time Per Frame: 0.10743451118469238
[INFO] percentage of bounding box in total image : 5.15
PREDICTIONS : [[1.44517852e-01 6.67881098e-03 2.30166105e-04 2.45430815e-03
  2.80040813e-02 1.08000249e-03 7.85215513e-01 2.69264645e-02
  6.06842382e-04 6.99613242e-04 3.58634584e-03]]
BEST CLASS INDICES [6]
[0.7

...

[INFO] total boxes: 0
[INFO] Processing Frame: 717
[INFO] Processing Speed: 19.144642490357622  FPS
[INFO] Time Per Frame: 0.05223393440246582
[INFO] total boxes: 0
[INFO] Processing Frame: 718
[INFO] Processing Speed: 19.597902980123166  FPS
[INFO] Time Per Frame: 0.0510258674621582
[INFO] total boxes: 0
[INFO] Processing Frame: 719
[INFO] Processing Speed: 19.374839478570966  FPS
[INFO] Time Per Frame: 0.05161333084106445
[INFO] total boxes: 0
[INFO] Processing Frame: 720
[INFO] Processing Speed: 19.56900911661239  FPS
[INFO] Time Per Frame: 0.0511012077331543
[INFO] percentage of bounding box in total image : 0.87
[INFO] total boxes: 1
[INFO] Processing Frame: 721
[INFO] Processing Speed: 16.16688380268118  FPS
[INFO] Time Per Frame: 0.06185483932495117
[INFO] total boxes: 0
[INFO] Processing Frame: 722
[INFO] Processing Speed: 19.850371043465092  FPS
[INFO] Time Per Frame: 0.05037689208984375
[INFO] percentage of bounding box in total image : 0.78
[INFO] total boxes: 1
[INFO] Proce

[INFO] total boxes: 0
[INFO] Processing Frame: 760
[INFO] Processing Speed: 14.231390937900802  FPS
[INFO] Time Per Frame: 0.0702672004699707
[INFO] total boxes: 0
[INFO] Processing Frame: 761
[INFO] Processing Speed: 15.414908083235941  FPS
[INFO] Time Per Frame: 0.06487226486206055
[INFO] total boxes: 0
[INFO] Processing Frame: 762
[INFO] Processing Speed: 16.923297907537865  FPS
[INFO] Time Per Frame: 0.05909013748168945
[INFO] percentage of bounding box in total image : 1.02
PREDICTIONS : [[6.31548895e-03 1.85416158e-03 2.13752889e-04 5.21252727e-02
  7.10391863e-03 5.56140338e-02 7.60268053e-01 2.39700746e-02
  4.43870401e-02 4.77914181e-02 3.56785836e-04]]
BEST CLASS INDICES [6]
[0.76026805]
Saikrishna_Dyavarasetti
[INFO] percentage of bounding box in total image : 0.44
[INFO] total boxes: 2
[INFO] Processing Frame: 763
[INFO] Processing Speed: 10.879234721800728  FPS
[INFO] Time Per Frame: 0.0919182300567627
[INFO] total boxes: 0
[INFO] Processing Frame: 764
[INFO] Processing Sp

[INFO] total boxes: 0
[INFO] Processing Frame: 814
[INFO] Processing Speed: 19.705908055157508  FPS
[INFO] Time Per Frame: 0.05074620246887207
[INFO] total boxes: 0
[INFO] Processing Frame: 815
[INFO] Processing Speed: 21.19491439775231  FPS
[INFO] Time Per Frame: 0.047181129455566406
[INFO] total boxes: 0
[INFO] Processing Frame: 816
[INFO] Processing Speed: 17.923916480773997  FPS
[INFO] Time Per Frame: 0.055791378021240234
[INFO] total boxes: 0
[INFO] Processing Frame: 817
[INFO] Processing Speed: 21.10947038899206  FPS
[INFO] Time Per Frame: 0.04737210273742676
[INFO] total boxes: 0
[INFO] Processing Frame: 818
[INFO] Processing Speed: 21.387179840194175  FPS
[INFO] Time Per Frame: 0.04675698280334473
[INFO] total boxes: 0
[INFO] Processing Frame: 819
[INFO] Processing Speed: 19.678726089547197  FPS
[INFO] Time Per Frame: 0.05081629753112793
[INFO] total boxes: 0
[INFO] Processing Frame: 820
[INFO] Processing Speed: 20.748063357638237  FPS
[INFO] Time Per Frame: 0.04819726943969726

[INFO] total boxes: 0
[INFO] Processing Frame: 874
[INFO] Processing Speed: 20.08439238821265  FPS
[INFO] Time Per Frame: 0.0497899055480957
[INFO] total boxes: 0
[INFO] Processing Frame: 875
[INFO] Processing Speed: 20.662919300250756  FPS
[INFO] Time Per Frame: 0.04839587211608887
[INFO] total boxes: 0
[INFO] Processing Frame: 876
[INFO] Processing Speed: 19.709056392761653  FPS
[INFO] Time Per Frame: 0.05073809623718262
[INFO] total boxes: 0
[INFO] Processing Frame: 877
[INFO] Processing Speed: 19.40154683047774  FPS
[INFO] Time Per Frame: 0.05154228210449219
[INFO] total boxes: 0
[INFO] Processing Frame: 878
[INFO] Processing Speed: 19.211285926943777  FPS
[INFO] Time Per Frame: 0.05205273628234863
[INFO] total boxes: 0
[INFO] Processing Frame: 879
[INFO] Processing Speed: 19.763477441394745  FPS
[INFO] Time Per Frame: 0.0505983829498291
[INFO] total boxes: 0
[INFO] Processing Frame: 880
[INFO] Processing Speed: 19.336520508411283  FPS
[INFO] Time Per Frame: 0.05171561241149902
[IN

[INFO] total boxes: 0
[INFO] Processing Frame: 930
[INFO] Processing Speed: 19.164586922054127  FPS
[INFO] Time Per Frame: 0.052179574966430664
[INFO] total boxes: 0
[INFO] Processing Frame: 931
[INFO] Processing Speed: 19.78519944148836  FPS
[INFO] Time Per Frame: 0.05054283142089844
[INFO] total boxes: 0
[INFO] Processing Frame: 932
[INFO] Processing Speed: 19.44535158115319  FPS
[INFO] Time Per Frame: 0.05142617225646973
[INFO] total boxes: 0
[INFO] Processing Frame: 933
[INFO] Processing Speed: 18.225010862952985  FPS
[INFO] Time Per Frame: 0.054869651794433594
[INFO] total boxes: 0
[INFO] Processing Frame: 934
[INFO] Processing Speed: 19.394369844264418  FPS
[INFO] Time Per Frame: 0.05156135559082031
[INFO] total boxes: 0
[INFO] Processing Frame: 935
[INFO] Processing Speed: 18.30966142241003  FPS
[INFO] Time Per Frame: 0.05461597442626953
[INFO] total boxes: 0
[INFO] Processing Frame: 936
[INFO] Processing Speed: 19.42787795693197  FPS
[INFO] Time Per Frame: 0.05147242546081543
[